In [49]:
import pandas as pd

births = pd.read_csv("birth_data.csv")
births.columns = ["Month", "Birth Rate"]
births.index = births["Month"]
births.drop(columns=["Month"], inplace=True)


jobs = pd.read_csv("Employment_no_SA.csv")
jobs = jobs.drop(["Month.1"], axis=1)
jobs.index = jobs.Month
jobs = jobs.drop(["Month"], axis=1)
jobs.columns = ["Employment"]

bus_rides = pd.read_csv("passengers_per_month.csv")
bus_rides.columns = ["Month", "Avg Bus Utilization"]
bus_rides.index = bus_rides["Month"]
bus_rides.drop(columns=["Month"], inplace=True)


population = pd.read_csv("population_data.csv")
air = pd.read_csv("air_quality_data.csv")
air["DateTime"] = pd.to_datetime(air["DateTime"])
air.set_index("DateTime", inplace=True)

air_month = pd.DataFrame(air.resample("BMS").mean()["AQI_Site"])
air_month.columns = ["AQI"]
air_month.index = pd.to_datetime(air_month.index)

Serieses = [births, jobs]
# Serieses = [births, jobs, bus_rides]
for s in Serieses:
    print(s.head(3))

            Birth Rate
Month                 
2006-01-01         390
2006-02-01         364
2006-03-01         417
            Employment
Month                 
1978-02-01        61.1
1978-03-01        61.1
1978-04-01        61.1


In [50]:
series_names = ["Employment", "Birth Rate"]

hist_data = pd.concat(Serieses, axis=1)
hist_data.dropna(inplace=True)
projection = pd.DataFrame(columns=hist_data.columns)
for s_name in series_names:
    hist_data[s_name + " delta"] = hist_data[s_name].diff()
hist_data.drop(hist_data.head(1).index, inplace=True)
hist_data.drop(hist_data.tail(1).index, inplace=True)

/home/metaten/.virtualenvs/govhack/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,Birth Rate,Employment,Employment delta,Birth Rate delta
2017-09-01,442.0,65.3,0.1,3.0
2017-10-01,422.0,65.4,0.1,-20.0
2017-11-01,350.0,65.5,0.1,-72.0


In [66]:
periods = 120
pred = pd.date_range(hist_data.tail(1).index.values[0], periods=periods, freq='M')
e_end: float = 70 
e_start: float = 65
def lerp(start, end, frac):
    return start + ((end - start) * frac)

def lerp_range(start, end, steps):
    return [lerp(start, end, x/(steps -1)) for x in range(steps)]

predictions = pd.DataFrame({"Date": pred, "Employment": lerp_range(e_start,e_end, periods)})

In [53]:
hist_data.tail(3)

,Birth Rate,Employment,Employment delta,Birth Rate delta
2017-09-01,442.0,65.3,0.1,3.0
2017-10-01,422.0,65.4,0.1,-20.0
2017-11-01,350.0,65.5,0.1,-72.0


In [94]:
hist_data["LS"] = pd.Series()
hist_data["LS"] = hist_data["LS"].fillna(0)
prev_row = hist_data.tail(1).copy(deep=True)
prev_row.drop(columns=["LS"], inplace=True)
predicted_series = "Employment"
series_names = ["Employment", "Birth Rate"]
projection = pd.DataFrame(columns=["Date"].append(series_names))
projection = []

for idx, row in predictions.iterrows():
    diff = row[predicted_series] - prev_row[predicted_series].values[0]
    new = prev_row.copy(deep=True)
    # print(new)
    new[predicted_series] = row[predicted_series]
    new.iloc[0][predicted_series + " delta"] = diff
    hist_data["LS"] = pd.Series()
    hist_data["LS"] = hist_data["LS"].fillna(0)
    for series in series_names:
        hist_data["LS"] += ((hist_data[series + " delta"] - new[series + " delta"].values[0])/
                           hist_data[series + " delta"].abs().mean()) ** 2
    # print(new)
    least = hist_data[hist_data["LS"] == hist_data["LS"].min()]
    # print(hist_data.tail(3))
    # print(least)
    new_proj = {"Date": row["Date"]}
    for series in series_names:
        if series != predicted_series:
            new[series] += least[series + " delta"].values[0]
        new_proj[series] = new[series].values[0]
    prev_row = new.copy(deep=True)
    print(new_proj)
    projection.append(new_proj, ignore_index=True)
        
    
    
    
    
    # projection.append()
projection

{'Date': Timestamp('2017-11-30 00:00:00'), 'Employment': 65.0, 'Birth Rate': 304.0}
{'Date': Timestamp('2017-12-31 00:00:00'), 'Employment': 65.04201680672269, 'Birth Rate': 232.0}
{'Date': Timestamp('2018-01-31 00:00:00'), 'Employment': 65.08403361344538, 'Birth Rate': 160.0}
{'Date': Timestamp('2018-02-28 00:00:00'), 'Employment': 65.12605042016807, 'Birth Rate': 88.0}
{'Date': Timestamp('2018-03-31 00:00:00'), 'Employment': 65.16806722689076, 'Birth Rate': 16.0}
{'Date': Timestamp('2018-04-30 00:00:00'), 'Employment': 65.21008403361344, 'Birth Rate': -56.0}
{'Date': Timestamp('2018-05-31 00:00:00'), 'Employment': 65.25210084033614, 'Birth Rate': -128.0}
{'Date': Timestamp('2018-06-30 00:00:00'), 'Employment': 65.29411764705883, 'Birth Rate': -200.0}
{'Date': Timestamp('2018-07-31 00:00:00'), 'Employment': 65.33613445378151, 'Birth Rate': -272.0}
{'Date': Timestamp('2018-08-31 00:00:00'), 'Employment': 65.3781512605042, 'Birth Rate': -344.0}
{'Date': Timestamp('2018-09-30 00:00:00'),

""


In [188]:
doo["LS"] = doo["LS"].fillna(0)
doo

,Birth Rate,Employment,Employment delta,Birth Rate delta,LS
2006-02-01,364.0,64.5,0.0,-26.0,0.0
2006-03-01,417.0,64.5,0.0,53.0,0.0
2006-04-01,374.0,64.6,0.1,-43.0,0.0
2006-05-01,361.0,64.7,0.1,-13.0,0.0
2006-06-01,360.0,64.8,0.1,-1.0,0.0
...,...,...,...,...,...
2017-07-01,441.0,65.2,0.1,-1.0,0.0
2017-08-01,439.0,65.2,0.0,-2.0,0.0
2017-09-01,442.0,65.3,0.1,3.0,0.0
2017-10-01,422.0,65.4,0.1,-20.0,0.0


In [101]:

doo["LS"] = ((doo["E delta"] - 61)/ doo["E delta"].mean()) ** 2 \
            + ((doo["B delta"] - 400)/doo["B delta"].mean()) ** 2 
doo[doo["LS"] == doo["LS"].min()]

,Birth Rate,Employment %,E delta,B delta,LS
2011-09-01,474.0,65.5,0.1,59.0,3.802516e+07


In [100]:
doo

,Birth Rate,Employment %,E delta,B delta,LS
1978-03-01,NaN,61.1,0.0,NaN,NaN
1978-04-01,NaN,61.1,0.0,NaN,NaN
1978-05-01,NaN,61.1,0.0,NaN,NaN
1978-06-01,NaN,61.0,-0.1,NaN,NaN
1978-07-01,NaN,61.0,0.0,NaN,NaN
...,...,...,...,...,...
2019-02-01,NaN,65.7,0.0,NaN,NaN
2019-03-01,NaN,65.8,0.1,NaN,NaN
2019-04-01,NaN,65.9,0.1,NaN,NaN
2019-05-01,NaN,66.0,0.1,NaN,NaN


In [ ]:
def dist(series, target):
    return ((series - target[series.nam])/series.mean()) ** 2
doo.apply(lambda :)

In [ ]:
from typing import List

pred = pd.DataFrame()

class Month:
    def __init__(self, y, z):
        y: float = y
        z: float = z
    
    # def distance(self, other: Month) -> float:
    #     


sep = Month(400, )



new_state = Month()